In [1]:
import json

file_path = "/kaggle/input/crime-dataset/qa_model_tanzania_crime.json"

# Load the JSON data
with open(file_path, 'r') as f:
    qa_data = json.load(f)

# Inspect the data
print(qa_data[:5])  # This will print the first 5 entries in the dataset


[{'context': 'Article 13 of the Constitution of Tanzania states that all persons are equal before the law and are entitled, without discrimination, to protection and equality before the law.', 'question': 'What does the Tanzanian Constitution say about equality before the law in criminal cases?', 'answer': 'All persons are equal before the law and are entitled to protection and equality before the law.', 'source': 'Article 13, Constitution of Tanzania'}, {'context': 'Article 30 of the Constitution limits the exercise of individual rights and freedoms if they infringe upon the rights and freedoms of others or affect public safety and order.', 'question': 'Can individual rights be limited in Tanzania if they threaten public safety?', 'answer': 'Yes, individual rights can be limited if they threaten public safety and order.', 'source': 'Article 30, Constitution of Tanzania'}, {'context': 'Article 31 of the Constitution discusses derogation from rights and freedoms in situations of emergen

In [2]:
import pandas as pd

df = pd.DataFrame(qa_data)

In [3]:
df.head()

,context,question,answer,source
0,Article 13 of the Constitution of Tanzania sta...,What does the Tanzanian Constitution say about...,All persons are equal before the law and are e...,"Article 13, Constitution of Tanzania"
1,Article 30 of the Constitution limits the exer...,Can individual rights be limited in Tanzania i...,"Yes, individual rights can be limited if they ...","Article 30, Constitution of Tanzania"
2,Article 31 of the Constitution discusses derog...,Can rights and freedoms be derogated in Tanzan...,"Yes, rights and freedoms can be derogated in s...","Article 31, Constitution of Tanzania"
3,Article 45 of the Constitution gives the Presi...,Does the President of Tanzania have the power ...,"Yes, the President has the power to pardon con...","Article 45, Constitution of Tanzania"
4,Article 28 of the Constitution imposes a duty ...,What duty does the Tanzanian Constitution impo...,Citizens have the duty to defend and protect t...,"Article 28, Constitution of Tanzania"


In [4]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size = 0.1, random_state = 42)

In [6]:
# Save the train and test sets
train_df.to_json('train_data.json', orient='records', lines=True)
test_df.to_json('test_data.json', orient='records', lines=True)

In [7]:
!pip install transformers datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 3.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.8/446.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 11.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import json
from datasets import load_dataset

# Load the dataset from JSON files
data_files = {
    "train": "train_data.json",
    "test": "test_data.json"
}

dataset = load_dataset("json", data_files=data_files)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 17 examples [00:00, 4262.76 examples/s]
Generating test split: 2 examples [00:00, 1101.01 examples/s]


In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer', 'source'],
        num_rows: 17
    })
    test: Dataset({
        features: ['context', 'question', 'answer', 'source'],
        num_rows: 2
    })
})


In [11]:
from transformers import LlamaTokenizerFast

# Load the LLaMA tokenizer
tokenizer = LlamaTokenizerFast.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2')

# Set the pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True, max_length=512)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Map: 100%|██████████| 2/2 [00:00<00:00, 309.12 examples/s]


In [13]:
from transformers import LlamaForCausalLM

# Load the LLaMA 3.1 8B model
model = LlamaForCausalLM.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2') 

/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:202: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:22<00:00,  5.72s/it]


In [28]:
!pip install torch-xla

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [40]:
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from transformers import TrainingArguments, Trainer

In [41]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    use_ipex=False,  # Disable Intel Extension for PyTorch
    fp16=False,  # Disable fp16 as it's not supported on TPU
    bf16=True,  # Use bfloat16 instead, which is supported on TPU
    tpu_num_cores=8,  # Number of TPU cores
)

In [42]:
def train_func(index):
    # Set the device
    device = xm.xla_device()
    
    # Initialize your model (replace with your actual model initialization)
    model = YourModelClass().to(device)
    
    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test']
    )
    
    # Train the model
    trainer.train()
    
    # Save the model
    xm.save(trainer.model.state_dict(), 'path_to_save_model.bin')

In [43]:
if __name__ == "__main__":
    xmp.spawn(train_func, nprocs=8)  # Start 8 processes, one for each TPU core

RuntimeError: Runtime is already initialized. Do not use the XLA device before calling xmp.spawn.

In [45]:
import os
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer

def train_func(index):
    # Set the device
    #device = xm.xla_device()
    
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2')
    model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2').to(device)
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        use_ipex=False,
        fp16=False,
        bf16=True,
        tpu_num_cores=8,
    )
    
    
    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
    )
    
    # Train the model
    trainer.train()
    
    # Save the model on the main process
    if xm.is_master_ordinal():
        trainer.save_model("./saved_model")
        tokenizer.save_pretrained("./saved_model")

def main():
    # Start the multiprocessing
    xmp.spawn(train_func, nprocs=8)

if __name__ == "__main__":
    main()

RuntimeError: Runtime is already initialized. Do not use the XLA device before calling xmp.spawn.

In [46]:
import os
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer

def train_func(index):
    # Set the device
    device = xm.xla_device()
    
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2')
    model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2').to(device)
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        use_ipex=False,
        fp16=False,
        bf16=True,
        tpu_num_cores=8,
    )
    
    # Load and tokenize your datasets here
    # This is where you should prepare your tokenized_datasets
    tokenized_datasets = prepare_datasets(tokenizer)
    
    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
    )
    
    # Train the model
    trainer.train()
    
    # Save the model on the main process
    if xm.is_master_ordinal():
        trainer.save_model("./saved_model")
        tokenizer.save_pretrained("./saved_model")

def prepare_datasets(tokenizer):
    # Implement your dataset preparation logic here
    # Make sure not to use any XLA operations in this function
    # Return a dictionary with 'train' and 'test' datasets
    pass

def main():
    # Start the multiprocessing
    xmp.spawn(train_func, nprocs=8)

if __name__ == "__main__":
    main()

RuntimeError: Runtime is already initialized. Do not use the XLA device before calling xmp.spawn.

In [47]:
import os
import torch
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer

def train_func(index):
    # Import XLA modules only inside the function
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl

    # Set the device
    device = xm.xla_device()
    
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2')
    model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/llama-3.1/transformers/8b/2').to(device)
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        use_ipex=False,
        fp16=False,
        bf16=True,
        tpu_num_cores=8,
    )
    
    # Load and tokenize your datasets here
    tokenized_datasets = prepare_datasets(tokenizer)
    
    # Create the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
    )
    
    # Train the model
    trainer.train()
    
    # Save the model on the main process
    if xm.is_master_ordinal():
        trainer.save_model("./saved_model")
        tokenizer.save_pretrained("./saved_model")

def prepare_datasets(tokenizer):
    # Implement your dataset preparation logic here
    # Make sure not to use any XLA operations in this function
    # Return a dictionary with 'train' and 'test' datasets
    pass

if __name__ == "__main__":
    # Import XLA multiprocessing module only when needed
    import torch_xla.distributed.xla_multiprocessing as xmp
    xmp.spawn(train_func, nprocs=8)

RuntimeError: Runtime is already initialized. Do not use the XLA device before calling xmp.spawn.